In [136]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Suteki_Da_Ne_(Piano_Version).mid to Suteki_Da_Ne_(Piano_Version).mid
User uploaded file "Suteki_Da_Ne_(Piano_Version).mid" with length 6814 bytes


In [78]:
!pip install mido

In [0]:
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam,SGD,Adagrad
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mido

In [0]:
midi = MidiFile('Suteki_Da_Ne_(Piano_Version).mid') 

In [0]:
#preprocess the midi file
notes=[]

time=float(0)
prev=float(0)
#The tracks attribute is a list of tracks. 
#Each track is a list of messages and meta messages, with the time attribute of each messages set to its delta time (in ticks)
for msg in midi:
	time += msg.time
	if not msg.is_meta: #easy to playback on port 
		#only interested in piano channel
		if msg.channel==0:
			if msg.type=='note_on':
				note=msg.bytes() 
				# only interested in the note and velocity. note message is in the form [type, note, velocity]
				note=note[1:3]
				note.append(time-prev)
				prev=time
				notes.append(note)  

In [162]:
notes

[[69, 77, 0.0],
 [69, 0, 0.424999575],
 [70, 64, 0.003571425000000017],
 [70, 0, 0.42499957499999996],
 [77, 64, 0.0035714249999999614],
 [77, 0, 0.4249995750000001],
 [67, 64, 0.0035714250000000725],
 [67, 0, 0.42499957499999996],
 [69, 64, 0.0035714250000000725],
 [69, 0, 0.4249995750000002],
 [76, 64, 0.0035714250000000725],
 [76, 0, 0.4249995750000002],
 [65, 64, 0.0035714250000000725],
 [65, 0, 0.4249995750000002],
 [72, 64, 0.0035714250000000725],
 [72, 0, 0.4249995750000002],
 [74, 77, 0.0035714250000000725],
 [74, 0, 0.4249995750000002],
 [64, 64, 0.0035714250000000725],
 [64, 0, 0.42499957499999974],
 [65, 64, 0.0035714249999996284],
 [65, 0, 0.4249995750000002],
 [72, 64, 0.0035714249999996284],
 [72, 0, 1.2821415749999998],
 [60, 64, 0.0035714249999996284],
 [64, 64, 0.0],
 [64, 0, 0.853570575],
 [60, 0, 0.0],
 [69, 80, 0.0035714249999996284],
 [69, 0, 1.7107125749999996],
 [76, 64, 0.0035714250000005165],
 [76, 0, 1.282141574999999],
 [76, 64, 0.0035714250000005165],
 [76, 

In [0]:
# need to scale notes
n=[]
for note in notes:
	note[0] = (note[0]-24)/88
	note[1] = note[1]/127
	n.append(note[2])
max_n = max(n) # scale based on the biggest time of any note
for note in notes:
	note[2] = note[2]/max_n

In [164]:
notes

[[0.5113636363636364, 0.6062992125984252, 0.0],
 [0.5113636363636364, 0.0, 0.12408759124086685],
 [0.5227272727272727, 0.5039370078740157, 0.0010427528675703145],
 [0.5227272727272727, 0.0, 0.12408759124086684],
 [0.6022727272727273, 0.5039370078740157, 0.0010427528675702985],
 [0.6022727272727273, 0.0, 0.12408759124086687],
 [0.48863636363636365, 0.5039370078740157, 0.0010427528675703308],
 [0.48863636363636365, 0.0, 0.12408759124086684],
 [0.5113636363636364, 0.5039370078740157, 0.0010427528675703308],
 [0.5113636363636364, 0.0, 0.1240875912408669],
 [0.5909090909090909, 0.5039370078740157, 0.0010427528675703308],
 [0.5909090909090909, 0.0, 0.1240875912408669],
 [0.4659090909090909, 0.5039370078740157, 0.0010427528675703308],
 [0.4659090909090909, 0.0, 0.1240875912408669],
 [0.5454545454545454, 0.5039370078740157, 0.0010427528675703308],
 [0.5454545454545454, 0.0, 0.1240875912408669],
 [0.5681818181818182, 0.6062992125984252, 0.0010427528675703308],
 [0.5681818181818182, 0.0, 0.12408

In [165]:
x = []
y = []
n_p = 20
for i in range(len(notes)-n_p):
	current = notes[i:i+n_p]
	next = notes[i+n_p]
	x.append(current)
	y.append(next)
print(x[1])
print(y[1])
x=np.array(x)  # convert to numpy arrays to pass it through model
y=np.array(y)
print(x[1])
print(y[1])


[[0.5113636363636364, 0.0, 0.12408759124086685], [0.5227272727272727, 0.5039370078740157, 0.0010427528675703145], [0.5227272727272727, 0.0, 0.12408759124086684], [0.6022727272727273, 0.5039370078740157, 0.0010427528675702985], [0.6022727272727273, 0.0, 0.12408759124086687], [0.48863636363636365, 0.5039370078740157, 0.0010427528675703308], [0.48863636363636365, 0.0, 0.12408759124086684], [0.5113636363636364, 0.5039370078740157, 0.0010427528675703308], [0.5113636363636364, 0.0, 0.1240875912408669], [0.5909090909090909, 0.5039370078740157, 0.0010427528675703308], [0.5909090909090909, 0.0, 0.1240875912408669], [0.4659090909090909, 0.5039370078740157, 0.0010427528675703308], [0.4659090909090909, 0.0, 0.1240875912408669], [0.5454545454545454, 0.5039370078740157, 0.0010427528675703308], [0.5454545454545454, 0.0, 0.1240875912408669], [0.5681818181818182, 0.6062992125984252, 0.0010427528675703308], [0.5681818181818182, 0.0, 0.1240875912408669], [0.45454545454545453, 0.5039370078740157, 0.001042

In [167]:
model=Sequential()
model.add(LSTM(512,input_shape=(20,3),return_sequences=True)) #np is 20*3
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True)) #return_sequences=False #return_sequences=False
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(3,activation="softmax")) #output=3

model.compile(loss="categorical_crossentropy",optimizer="RMSprop",metrics=["accuracy"])
model.fit(x,y,epochs=1000,batch_size=200,validation_split=0.1)

Train on 955 samples, validate on 107 samples
Epoch 1/1000
955/955 [==============================] - 10s 11ms/step - loss: 0.9635 - acc: 0.5780 - val_loss: 0.9584 - val_acc: 0.9346
Epoch 2/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8618 - acc: 0.8597 - val_loss: 0.8934 - val_acc: 0.9346
Epoch 3/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8201 - acc: 0.8597 - val_loss: 0.8804 - val_acc: 0.9346
Epoch 4/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8194 - acc: 0.8597 - val_loss: 0.8665 - val_acc: 0.9346
Epoch 5/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8126 - acc: 0.8597 - val_loss: 0.9196 - val_acc: 0.9346
Epoch 6/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8233 - acc: 0.8597 - val_loss: 0.8896 - val_acc: 0.9346
Epoch 7/1000
955/955 [==============================] - 1s 1ms/step - loss: 0.8230 - acc: 0.8597 - val_loss: 0.8780 - val_acc: 0.9346
Epoch 8/1000
9

In [168]:
seed=notes[0:n_p]
x=seed
x=np.expand_dims(x, axis=0)
print(x)
predict=[]
for i in range(2000):
	p=model.predict(x)
	x=np.squeeze(x) #squeezed to concateneate
	x=np.concatenate((x, p))
	x=x[1:]
	x=np.expand_dims(x, axis=0) #expanded to roll back
	p=np.squeeze(p)
	predict.append(p)
  

[[[0.51136364 0.60629921 0.        ]
  [0.51136364 0.         0.12408759]
  [0.52272727 0.50393701 0.00104275]
  [0.52272727 0.         0.12408759]
  [0.60227273 0.50393701 0.00104275]
  [0.60227273 0.         0.12408759]
  [0.48863636 0.50393701 0.00104275]
  [0.48863636 0.         0.12408759]
  [0.51136364 0.50393701 0.00104275]
  [0.51136364 0.         0.12408759]
  [0.59090909 0.50393701 0.00104275]
  [0.59090909 0.         0.12408759]
  [0.46590909 0.50393701 0.00104275]
  [0.46590909 0.         0.12408759]
  [0.54545455 0.50393701 0.00104275]
  [0.54545455 0.         0.12408759]
  [0.56818182 0.60629921 0.00104275]
  [0.56818182 0.         0.12408759]
  [0.45454545 0.50393701 0.00104275]
  [0.45454545 0.         0.12408759]]]


In [0]:
# unrolling back from conversion 
for a in predict:
	a[0] = int(88*a[0] + 24)
	a[1] = int(127*a[1])
	a[2] *= max_n
	# reject values out of range  (note[0]=24-102)(note[1]=0-127)(note[2]=0-__)
	if a[0] < 24:  
		a[0] = 24
	elif a[0] > 102:
		a[0] = 102
	if a[1] < 0:    
		a[1] = 0
	elif a[1] > 127:
		a[1] = 127
	if a[2] < 0:     
		a[2] = 0

In [170]:
from mido import MidiFile, MidiTrack, Message
#saving track from bytes data
m=MidiFile()
track=MidiTrack()
m.tracks.append(track)

for note in predict:
	#147 means note_on
	note=np.insert(note, 0, 147)
	bytes=note.astype(int)
	print(note)
	msg = Message.from_bytes(bytes[0:3]) 
	time = int(note[3]/0.001025) # to rescale to midi's delta ticks. arbitrary value
	msg.time = time
	track.append(msg)

m.save('Ai_song.mid')


[1.4700000e+02 6.7000000e+01 6.4000000e+01 5.3772866e-03]
[147.          92.           0.           0.74633193]
[1.4700000e+02 6.8000000e+01 6.0000000e+01 6.1497893e-02]
[147.        82.         0.         1.164013]
[1.4700000e+02 6.4000000e+01 6.7000000e+01 8.3275366e-04]
[1.470000e+02 6.700000e+01 6.300000e+01 1.428513e-03]
[147.         75.          0.          1.4330724]
[1.470000e+02 1.020000e+02 0.000000e+00 3.459759e-04]
[1.4700000e+02 6.5000000e+01 6.7000000e+01 2.4868783e-03]
[147.         65.          0.          1.7924477]
[1.470000e+02 7.000000e+01 5.800000e+01 2.819811e-02]
[147.         79.          4.          1.1532999]
[1.4700000e+02 6.9000000e+01 6.1000000e+01 1.9107182e-02]
[147.         100.           0.           0.45790693]
[1.4700000e+02 6.7000000e+01 6.4000000e+01 1.4668623e-03]
[147.         73.          0.          1.4841338]
[147.          60.          47.           0.73050207]
[147.          88.           0.           0.91861445]
[1.4700000e+02 6.9000000e+01

In [0]:
from google.colab import files
files.download('Ai_song.mid')